In [6]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.feature
import librosa.display
import glob
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical
import tensorflow as tf

In [7]:
def extract_info():
    list_features = []
    list_labels = []
    
    animals = ['cat', 'dog']
    for animal in animals:
        audio = glob.glob('barkmeow/'+animal+'/*.wav')
        for audio_file in audio:
            y, sr = librosa.load(audio_file)
            mfcc1 = librosa.feature.mfcc(y)
            mfcc2 = mfcc1 / np.amax(np.absolute(mfcc1))
            features = np.ndarray.flatten(mfcc2)[:25]
            list_features.append(features)
            list_labels.append(animal)
            
    label_uniq_ids, label_row_ids = np.unique(list_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int, copy=False)
    onehot_labels = to_categorical(label_row_ids, len(label_uniq_ids))
    return np.stack(list_features), onehot_labels

In [8]:
features, labels = extract_info()

In [10]:
split_ratio = 0.8

alldata = np.column_stack((features, labels))

np.random.shuffle(alldata)
splitidx = int(len(alldata) * split_ratio)
train, test = alldata[:splitidx,:], alldata[splitidx,:]

train_input = train[:,:-2]
train_labels = train[:,-2:]

test_input = train[:,:-2]
test_labels = train[:,-2:]

In [31]:
model = Sequential()

model.add(Dense(600, input_dim = (np.shape(train_input)[1])))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 600)               15600     
_________________________________________________________________
activation_14 (Activation)   (None, 600)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 1202      
_________________________________________________________________
activation_15 (Activation)   (None, 2)                 0         
Total params: 16,802
Trainable params: 16,802
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [33]:
model.fit(train_input, train_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 0s 62ms/step - loss: 0.6912 - accuracy: 0.5354 - val_loss: 0.6239 - val_accuracy: 0.7812
Epoch 2/10
4/4 [==============================] - 0s 26ms/step - loss: 0.6531 - accuracy: 0.6406 - val_loss: 0.5820 - val_accuracy: 0.8125
Epoch 3/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6221 - accuracy: 0.7385 - val_loss: 0.5571 - val_accuracy: 0.7188
Epoch 4/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6317 - accuracy: 0.6240 - val_loss: 0.5221 - val_accuracy: 0.7812
Epoch 5/10
4/4 [==============================] - 0s 21ms/step - loss: 0.6060 - accuracy: 0.7500 - val_loss: 0.4991 - val_accuracy: 0.8438
Epoch 6/10
4/4 [==============================] - 0s 24ms/step - loss: 0.5714 - accuracy: 0.7375 - val_loss: 0.4785 - val_accuracy: 0.8125
Epoch 7/10
4/4 [==============================] - 0s 17ms/step - loss: 0.5732 - accuracy: 0.7500 - val_loss: 0.4619 - val_accuracy: 0.7812
Epoch 8/10
4/4 [===========

In [26]:
loss, acc = model.evaluate(test_input, test_labels, batch_size=32)

print("Valeur du Loss: %.4f, et valeur de l'accuracy: %4f" % (loss, acc))

5/5 [==============================] - 0s 0s/step - loss: 0.5137 - accuracy: 0.7875
Valeur du Loss: 0.5137, et valeur de l'accuracy: 0.787500


In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]

tflite_model = converter.convert()
open("converted_model_latency.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\thanu\AppData\Local\Temp\tmpbyxlxffa\assets


20240